In [1]:
import pandas as pd
from countryinfo import CountryInfo
import pypopulation
import plotly.graph_objects as go

In [2]:
countries = pd.read_csv('countries.csv')

In [3]:
#https://data.worldbank.org/indicator/NY.GDP.MKTP.CD
GDP = pd.read_excel('GDP_2021.xlsx')

In [4]:
def add_code(df_input):
        def temp(row):
            input_country =  row.country
            country = CountryInfo(input_country)
            try: 
                code = country.alt_spellings()
                code = code[0]
                return code
            except: 
                return 'Code not found'
  
        df_output = df_input.assign(code = df_input.apply(temp, axis = 1))
        return df_output

In [5]:
def add_capital(df_input):
        def temp(row):
            input_country =  row.country
            country = CountryInfo(input_country)
            try: 
                capital = country.capital()
                return capital
            except: 
                return 'Capital not found'
  
        df_output = df_input.assign(capital = df_input.apply(temp, axis = 1))
        return df_output

In [6]:
def add_language(df_input):
        def temp(row):
            input_country =  row.country
            country = CountryInfo(input_country)
            try: 
                language = country.languages()
                language = list(map(lambda x: x.upper(), language))
                return language
            except: 
                return 'Language not found'
  
        df_output = df_input.assign(language = df_input.apply(temp, axis = 1))
        return df_output

In [7]:
def add_population(df_input):
        def temp(row):
            input_country =  row.country
            country = CountryInfo(input_country)
            try: 
                code =  country.alt_spellings()[0]
                population = pypopulation.get_population(code)
                population = '{:,}'.format(population).replace(',','.')
                return population
            except: 
                return 'Population not found'
  
        df_output = df_input.assign(population = df_input.apply(temp, axis = 1))
        return df_output

In [8]:
def add_GDP_2021(df_input):
        def temp(row):
            input_country =  row.country
            try: 
                for i in range (len(GDP)):
                    if input_country == GDP['Country Name'][i]:
                        GDP_2021 = GDP[2021][i]
                return GDP_2021
            except: 
                return 'GDP 2021 not found'
  
        df_output = df_input.assign(GDP_2021_USD = df_input.apply(temp, axis = 1))
        return df_output

In [9]:
countries = add_code(countries)
countries = add_capital(countries)
countries = add_language(countries)
countries = add_population(countries)
countries = add_GDP_2021(countries)

In [10]:
countries.head()

,country,code,capital,language,population,GDP_2021_USD
0,Afghanistan,AF,Kabul,"[PS, UZ, TK]",38.041.754,14786861638.453501
1,Albania,AL,Tirana,[SQ],2.854.191,18255787479.184601
2,Algeria,DZ,Algiers,[AR],43.053.054,163044443983.759003
3,American Samoa,AS,Pago Pago,"[EN, SM]",55.312,709000000.0
4,Andorra,Code not found,Capital not found,Language not found,Population not found,3330281523.53915


In [12]:
#https://medium.com/analytics-vidhya/plotly-for-geomaps-bb75d1de189f
#https://plotly.com/python/reference/layout/#layout-title-pad
### It have a bug
'''
fig = go.Figure(go.Scattergeo( locations = countries['code'],
z = countries['capital']

))
fig.update_geos(projection_type = "natural earth", showcountries = True, countrycolor="Black", showlakes = False)
fig.update_layout(title='World map', font=dict(size=18) , title_y=0.5, title_x=0.2, height=300, margin={"r":0,"t":20,"l":0,"b":20})
fig.show()'''

'\nfig = go.Figure(go.Scattergeo( locations = countries[\'code\'],\nz = countries[\'capital\']\n\n))\nfig.update_geos(projection_type = "natural earth", showcountries = True, countrycolor="Black", showlakes = False)\nfig.update_layout(title=\'World map\', font=dict(size=18) , title_y=0.5, title_x=0.2, height=300, margin={"r":0,"t":20,"l":0,"b":20})\nfig.show()'